https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):

        # forward
        pred = model(X)
        loss = loss_fn(pred,y)

        # backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 200 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [9]:
def test_loop(dataloader, model, loss_fn,):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0,0

    with torch.no_grad():
        for X,y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
epochs = 10

for i in range(epochs):
    print(f"Epochs {i+1} ----------------")
    train_loop(train_dataloader,model,loss_fn,optimizer)
    test_loop(test_dataloader,model,loss_fn)

print("Finished!")

Epochs 1 ----------------
loss: 2.150266 [    0/60000]
loss: 2.079705 [12800/60000]
loss: 2.034965 [25600/60000]
loss: 2.006636 [38400/60000]
loss: 1.931635 [51200/60000]
Test Error: 
 Accuracy: 51.7%, Avg loss: 1.851853 

Epochs 2 ----------------
loss: 1.890243 [    0/60000]
loss: 1.740260 [12800/60000]
loss: 1.658877 [25600/60000]
loss: 1.642060 [38400/60000]
loss: 1.575693 [51200/60000]
Test Error: 
 Accuracy: 58.9%, Avg loss: 1.487102 

Epochs 3 ----------------
loss: 1.560021 [    0/60000]
loss: 1.375647 [12800/60000]
loss: 1.319981 [25600/60000]
loss: 1.340931 [38400/60000]
loss: 1.305916 [51200/60000]
Test Error: 
 Accuracy: 62.6%, Avg loss: 1.234853 

Epochs 4 ----------------
loss: 1.316230 [    0/60000]
loss: 1.131491 [12800/60000]
loss: 1.104521 [25600/60000]
loss: 1.158591 [38400/60000]
loss: 1.136991 [51200/60000]
Test Error: 
 Accuracy: 64.4%, Avg loss: 1.078474 

Epochs 5 ----------------
loss: 1.153457 [    0/60000]
loss: 0.971256 [12800/60000]
loss: 0.971000 [25600/60

# saving loading model

In [13]:
import torch
import torchvision.models as models

In [15]:
model = models.vgg16(pretrained=True)

In [ ]:
# weights only
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

In [ ]:
# whole model
torch.save(model, "model.pth")

torch.load("model.pth")